# 1) Données

In [1]:
import pandas as pd
import numpy as np
import csv
import os

# calcul de la position dans le roulement d'un poste en fonction de la semaine
def rFunction(s,p):
    return ((p-s)%24)

# calcul du numéro de semaine correspondant au jour j
def sFunction(j):
    return j//7

# calcul du jour non travaillé pour une semaine donnée d'un opérateur à 80%
# en fonction du jour non travaillé la première semaine
def delta_i(s,d_i):
    return (d_i+s)%5

# operateurs + competences
dataOperateurs = pd.read_csv('./data/operateurs.csv',sep = ";" )

cardO = len(dataOperateurs.axes[0])
colsDataOperateurs = len(dataOperateurs.axes[1])
nomsOperateurs = dataOperateurs.iloc[0:cardO,0:3]
competencesOperateurs = dataOperateurs.iloc[0:cardO,3:(colsDataOperateurs+1)]

# postes (intitulés + rho)
dataPostes = pd.read_csv('./data/postes.csv',sep = ";" )
cardP = len(dataPostes.axes[0])
nomsPostes = dataPostes.iloc[0:cardO,1]
rho = [0,1,1,1,0,1,1,1,0,1,1,1,0,1,1,1,0,1,1,1,0,1,1,1]

# le nombre de semaines <=> nombre de postes <=> nombre d'opérateurs
#cardS = cardO
cardS = 2
cardJ = 7*cardS
cardR = cardO

# kappa_ip (opérateur possède la compétence pour le poste)
#kappa = pd.read_csv('./data/kappa_ip.csv',sep = ";" )
kappa = [
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1,0,1,1,1,0,0,0],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,0,0,1,1,1,0,1,1,1,1,1,1,0,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1,0,1,1,1,0,0,0],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1,0,1,1,1,0,0,0],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1,0,1,1,1,0,0,0]]

# sigma_pp' (postes sur le même créneau horaire)
#sigma = pd.read_csv('./data/sigma_pp.csv',sep = ";" )
sigma= [[1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
[0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1],
[0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0],
[0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1],
[0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
[0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0],
[0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0],
[0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0],
[0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
[0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1],
[0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0],
[0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1],
[0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0],
[0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0],
[0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0],
[0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0],
[0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0],
[0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1],
[0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0],
[0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0],
[0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0],
[0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1]]

# delta_ij (est-ce qu'une personne à 80% travaille le jour j)
#delta = pd.read_csv('./data/delta_ij.csv',sep = ";" )
# d[i] représente le jour non travaillé de l'opérateur i la première semaine, d[i]=-1 si i est à temps plein
d = [3,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1]
delta = np.full((cardO, cardJ), 1)
for i in range(0,cardO):
    if d[i] != -1:
        d_i = d[i]
        for s in range(0, cardS):
            d_is = delta_i(s,d_i) #rang du jour non travaillé la semaine s
            j_s = 7*s + d_is      #numéro du jour non travaillé
            delta[i][j_s] = 0

# o_j (jour ouvrés)
o = np.zeros(cardJ)
for j in range(0,len(o)):
    if (j%7 <= 4):
        o[j] = 1

# H_s (jours de chaque semaine)
H = []
for i in range(0,cardS):
    H.append(np.zeros(7))
for i in range(0,cardS):
    for j in range(0,7):
        H[i][j]=(i*7 + j)

#print(nomsOperateurs)
#print(competencesOperateurs)
#print(nomsPostes)
#print(rho)
#print(kappa)
#print(sigma)
#print(delta)
#print(o)
#print(H[1])


# 2) Modèle mathématique


## 2.1) Variables

In [2]:
from pulp import *

# x_ijp = 1 si l'opérateur prend en charge le poste p le jour j
x = {} 
for i in range(0,cardO):
    for j in range(0,cardJ): 
        for p in range(0,cardP): 
            x[i,j,p]=LpVariable(f"x({i,j,p})", cat=LpBinary)

# y_ir = 1 si l'opérateur i est position r dans le roulement
y = {} 
for i in range(0,cardO):
    for r in range(0,cardR):    
        y[i,r]=LpVariable(f"y({i,r})", cat=LpBinary)
        
# z_isp = 1 si l'opérateur est affecté au poste p la semaine s
z = {} 
for i in range(0,cardO):
    for s in range(0,cardS): 
        for p in range(0,cardP): 
            z[i,s,p]=LpVariable(f"z({i,s,p})", cat=LpBinary)

# u_is = 1 si l'opérateur i est affecté à un poste différent de celui prévu la semaine s
u = {} 
for i in range(0,cardO):
    for s in range(0,cardS):    
        u[i,s]=LpVariable(f"u({i,s})", cat=LpBinary)

print("nb de variables x_ijp = ", len(x))
print("nb de variables y_ir  = ", len(y))
print("nb de variables z_isp = ", len(z))
print("nb de variables u_ir  = ", len(u))
print()
print("nb total de variables = ", len(x)+len(y)+len(z)+len(u))

nb de variables x_ijp =  8064
nb de variables y_ir  =  576
nb de variables z_isp =  1152
nb de variables u_ir  =  48

nb total de variables =  9840


## 2.2) Création du modèle avec PuLP

In [3]:
prob = LpProblem("ModelPlanning", LpMinimize)

prob += sum(u[i,s] for i in range(0,cardO) for s in range(0,cardS)), "obj"

## 2.2) Contraintes

In [4]:
#contrainte 1 Un opérateur ne peut prendre un poste que s'il est compétent et disponible sur un jour ouvré

for i in range(0,cardO):
    for j in range(0,cardJ):
        for p in range(0,cardP):
            prob += (x[i,j,p] <= delta[i][j]*kappa[i][p]*o[j], f"C1_{i,j,p}")

#contrainte 2 Un opérateur ne peut être affecté à un poste une semaine que s'il en possède la compétence

for i in range(0,cardO):
    for s in range(0,cardS):
        for p in range(0,cardP):
            prob += (z[i,s,p] <= kappa[i][p], f"C2_{i,s,p}")

#contrainte 3 Chaque jour, l'opérateur est affecté à son poste de la semaine (s'il n'est pas rouleur)
for i in range(0,cardO):
    for s in range(0,cardS):
        for j in H[s]:
            for p in range(0,cardP):
                prob += x[i,j,p] <= z[i,s,p] + sum((1 - rho[p2])*z[i,s,p2] for p2 in range(0,cardP)), f"C3_{i,s,j,p}"

#contrainte 4 (page 8) Chaque jour, un opérateur est affecté à au plus un poste #MODIF

for i in range(0,cardO):
    for j in range(0,cardJ):
        if o[j] == 1 and delta[i][j] == 1:
            prob += sum(x[i,j,p] for p in range(0,cardP)) == 1, f"C4_{i,j,p}"

#contrainte 5 (page 8) Chaque jour, chaque poste est assuré par un opérateur #MODIF
for p in range(0,cardP):
    for j in range(0,cardJ):
        if o[j] == 1 and rho[p] == 1:
            prob += sum(x[i,j,p] for i in range(0,cardO)) == 1, f"C5_{i,j,p}"
            
#contrainte 5.2 BONUS Chaque jour, un poste roulant est occupé par au plus une personne
for p in range(0,cardP):
    for j in range(0,cardJ):
        if o[j] == 1 and rho[p] == 0:
            prob += sum(x[i,j,p] for i in range(0,cardO)) <= 1, f"C52_{i,j,p}"

#contrainte 6 (page 9) Chaque semaine, chaque opérateur est affecté à un poste

for i in range(0,cardO):
    for s in range(0,cardS):
        prob += sum(z[i,s,p] for p in range(0,cardP)) == 1, f"C6_{i,s,p}"

#contrainte 7 (page 9) Chaque semaine, chaque poste est assuré par un opérateur

for p in range(0,cardP):
    for s in range(0,cardS):
        prob += sum(z[i,s,p] for i in range(0,cardO)) == 1, f"C7_{i,s,p}"

#contrainte 8 (page 10) Chaque opérateur est affecté à une position dans le roulement

for i in range(0,cardO):
    prob += sum(y[i,r] for r in range(0,cardR)) == 1, f"C8_{i,r}"

#contrainte 9 (page 10) Chaque position dans le roulement est associée à un opérateur

for r in range(0,cardR):
    prob += sum(y[i,r] for i in range(0,cardO)) == 1, f"C9_{i,r}"

#contrainte 10 (page 11) Un opérateur peut changer de poste si l'horaire de ce dernier est compatible

for i in range(0,cardO):
    for s in range(0,cardS):
        for p in range(0,cardP):
            prob += z[i,s,p] <= sum(sigma[p][p2]*y[i,rFunction(s,p2)] for p2 in range(0,cardP)), f"C10_{i,s,p}"

#contrainte 11 (page 11) Lorsqu'un jour un opérateur est affecté à un poste A alors qu'il est affecté à un poste B pour cette semaine, cela augmente son score d'insatisfaction
for i in range(0,cardO):
    for s in range(0,cardS):
        for p in range(0,cardP):
            prob += u[i,s] >= (kappa[i][p]*rho[p]*y[i,rFunction(s,p)] - z[i,s,p]), f"C11_{i,s,p}"


## 2.4) Résolution du problème

In [5]:
import time

prob.writeLP("Model.lp")

start = time.time() 
prob.solve(CPLEX_PY())
end = time.time()

duree = end - start
print("Temps de résolution : " + str(duree))

Version identifier: 22.1.0.0 | 2022-03-09 | 1a383f8ce
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
MIP Presolve eliminated 12318 rows and 2748 columns.
MIP Presolve modified 1428 coefficients.
Reduced MIP has 7888 rows, 7092 columns, and 65082 nonzeros.
Reduced MIP has 7092 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.06 sec. (38.95 ticks)
Found incumbent of value 48.000000 after 0.17 sec. (92.35 ticks)
Probing changed sense of 288 constraints.
Probing time = 0.16 sec. (47.47 ticks)
Tried aggregator 1 time.
Detecting symmetries...
MIP Presolve eliminated 300 rows and 288 columns.
Reduced MIP has 7588 rows, 6804 columns, and 64218 nonzeros.
Reduced MIP has 6804 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.18 sec. (110.56 ticks)
Probing time = 0.02 sec. (8.07 ticks)
Clique table members: 27395.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up

## 2.5) Extraction de la solution

In [6]:
print("objectif =", value(prob.objective))

objectif = 0.0


## 2.6) Vérification de la validité de la solution

In [7]:

#Test 1 : 
test1 = True
for i in range(0,cardO):
    for j in range(0,cardJ):
        for p in range(0,cardP):
            if (x[i,j,p].varValue > delta[i][j]*kappa[i][p]*o[j]):
                test1 = False
                    
#Test 2 : 
test2 = True
for i in range(0,cardO):
    for s in range(0,cardS):
        for p in range(0,cardP):
            if (z[i,s,p].varValue > kappa[i][p]):
                test2 = False

#Test 3 : 
test3 = True
for i in range(0,cardO):
    for s in range(0,cardS):
        for j in H[s]:
            for p in range(0,cardP):
                if x[i,j,p].varValue > z[i,s,p].varValue + sum((1 - rho[p2])*z[i,s,p2].varValue for p2 in range(0,cardP)):
                    test3 = False

                    
#Test 4 : 
test4 = True
for i in range(0,cardO):
    for j in range(0,cardJ):
        if o[j] == 1 and delta[i][j] == 1:
            if sum(x[i,j,p].varValue for p in range(0,cardP)) != 1:
                test4 = False
                    
#Test 5 : 
test5 = True
for p in range(0,cardP):
    for j in range(0,cardJ):
        if o[j] == 1 and rho[p] == 1:
            if sum(x[i,j,p].varValue for i in range(0,cardO)) != 1:
                test5 = False
                    
#Test 6 : 
test6 = True
for i in range(0,cardO):
    for s in range(0,cardS):
        if sum(z[i,s,p].varValue for p in range(0,cardP)) != 1:
            test6 = False
                    
#Test 7 : 
test7 = True
for p in range(0,cardP):
    for s in range(0,cardS):
        if sum(z[i,s,p].varValue for i in range(0,cardO)) != 1:
            test7 = False

#Test 8 : 
test8 = True
for i in range(0,cardO):
    if sum(y[i,r].varValue for r in range(0,cardR)) != 1:
        test9 = False
                    
#Test 9 : 
test9 = True
for r in range(0,cardR):
    if sum(y[i,r].varValue for i in range(0,cardO)) != 1:
        test9 = False
                    
#Test 10 : 
test10 = True
for i in range(0,cardO):
    for s in range(0,cardS):
        for p in range(0,cardP):
            if z[i,s,p].varValue > sum(sigma[p][p2]*y[i,rFunction(s,p2)].varValue for p2 in range(0,cardP)):
                test10 = False
                    
#Test 11 : 
test11 = True
for i in range(0,cardO):
    for s in range(0,cardS):
        for p in range(0,cardP):
            if u[i,s].varValue < (kappa[i][p]*rho[p]*y[i,rFunction(s,p)].varValue - z[i,s,p].varValue):
                test11 = False
                
if test1:
    print('Test 1 OK')
else:
    print('Test 1 FAIL')
if test2:
    print('Test 2 OK')
else:
    print('Test 2 FAIL')
if test3:
    print('Test 3 OK')
else:
    print('Test 3 FAIL')
if test4:
    print('Test 4 OK')
else:
    print('Test 4 FAIL')
if test5:
    print('Test 5 OK')
else:
    print('Test 5 FAIL')
if test6:
    print('Test 6 OK')
else:
    print('Test 6 FAIL')
if test7:
    print('Test 7 OK')
else:
    print('Test 7 FAIL')
if test8:
    print('Test 8 OK')
else:
    print('Test 8 FAIL')
if test9:
    print('Test 9 OK')
else:
    print('Test 9 FAIL')
if test10:
    print('Test 10 OK')
else:
    print('Test 10 FAIL')
if test11:
    print('Test 11 OK')
else:
    print('Test 11 FAIL')

Test 1 OK
Test 2 OK
Test 3 OK
Test 4 OK
Test 5 OK
Test 6 OK
Test 7 OK
Test 8 OK
Test 9 OK
Test 10 OK
Test 11 OK


In [8]:
print("uis")
for s in range(0,cardS):
    for i in range(0,cardO):
        if(u[i,s].varValue == 1):
            print("u[",i,",",s,"] = 1")

print('Affectation aux roulements')
for i in range(0,24):
    for r in range(0,24):
        if(y[i,r].varValue == 1):
            print('y[',i,',',r,'] == 1')

print()
print('Affectations hebdomadaires')
for s in range(0,cardS):
    print('semaine',s)
    for i in range(0,cardO):
        for p in range(0,cardP):
            if z[i,s,p].varValue==1:
                print('semaine',s,',',i,'affecté au poste',p)
                
print('======================================================')

for p in range(0,cardP):
    print()
    if rho[p] == 1:
        print('poste',p,'non rouleur')
    else:
        print('poste',p,'rouleur')
    for j in range(0,cardJ):
        for i in range(0,cardO):
            if (x[i,j,p].varValue == 1):
                print(i,'occupe le poste',p,'le jour',j)
                
print('======================================================')

                
for i in range(0,cardO):
    print()
    print('planning de l\'opérateur',i)
    for j in range(0,cardJ):
        for p in range(0,cardP):
            if (x[i,j,p].varValue == 1):
                print('poste',p,'le jour',j)

uis
Affectation aux roulements
y[ 0 , 14 ] == 1
y[ 1 , 2 ] == 1
y[ 2 , 0 ] == 1
y[ 3 , 13 ] == 1
y[ 4 , 5 ] == 1
y[ 5 , 19 ] == 1
y[ 6 , 9 ] == 1
y[ 7 , 16 ] == 1
y[ 8 , 21 ] == 1
y[ 9 , 1 ] == 1
y[ 10 , 12 ] == 1
y[ 11 , 17 ] == 1
y[ 12 , 3 ] == 1
y[ 13 , 7 ] == 1
y[ 14 , 18 ] == 1
y[ 15 , 15 ] == 1
y[ 16 , 11 ] == 1
y[ 17 , 8 ] == 1
y[ 18 , 22 ] == 1
y[ 19 , 6 ] == 1
y[ 20 , 10 ] == 1
y[ 21 , 23 ] == 1
y[ 22 , 20 ] == 1
y[ 23 , 4 ] == 1

Affectations hebdomadaires
semaine 0
semaine 0 , 0 affecté au poste 14
semaine 0 , 1 affecté au poste 2
semaine 0 , 2 affecté au poste 0
semaine 0 , 3 affecté au poste 13
semaine 0 , 4 affecté au poste 5
semaine 0 , 5 affecté au poste 19
semaine 0 , 6 affecté au poste 9
semaine 0 , 7 affecté au poste 16
semaine 0 , 8 affecté au poste 21
semaine 0 , 9 affecté au poste 1
semaine 0 , 10 affecté au poste 12
semaine 0 , 11 affecté au poste 17
semaine 0 , 12 affecté au poste 3
semaine 0 , 13 affecté au poste 7
semaine 0 , 14 affecté au poste 18
semaine 0 ,

# Extraction des données au format CSV

In [9]:
import csv
with open('res.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    jours = [""]
    for s in range(0,cardS):
        jours.extend(["lundi","mardi","mercredi","jeudi","vendredi","samedi","dimanche"])
    writer.writerow(jours)
    
    dates = [""]
    for j in range(0,cardJ):
        dates.append("jour "+str(j))
    writer.writerow(dates)
    
    for i in range(0,cardO):
        planningPerso = [nomsOperateurs.iloc[i,1] + " " + nomsOperateurs.iloc[i,2]]
        for j in range(0,cardJ):
            found = False
            for p in range(0,cardP):
                if x[i,j,p].varValue == 1:
                    found = True
                    planningPerso.append(nomsPostes.iloc[p])
            if not found:
                if o[j] == 0:
                    planningPerso.append("")
                else:
                    planningPerso.append("RA")
        writer.writerow(planningPerso)